In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

In [8]:
from models.base import Base
base = Base(16, 128)
samples, base_log_det = base.sample_and_log_prob(128)

In [10]:
samples.shape

torch.Size([128, 8])

## models.base.Base & models.bijectors.CircularShift Test

Problem: nflow generate samples whose require_grad=False

In [34]:
from models.bijectors import CircularShift
from torch.nn import Parameter
samples.requires_grad=True
shift = Parameter(
    torch.rand(size=(1, )))
circular_shift = CircularShift(shift, -torch.pi, torch.pi)
shift_result, shift_logdet = circular_shift(samples)
shift_mean = shift_result.sum()
shift_mean.backward()
print(samples.grad)

tensor([[1.0010, 1.0010, 1.0010,  ..., 1.0010, 1.0010, 1.0010],
        [1.0010, 1.0010, 1.0010,  ..., 1.0010, 1.0010, 1.0010],
        [1.0010, 1.0010, 1.0010,  ..., 1.0010, 1.0010, 1.0010],
        ...,
        [1.0010, 1.0010, 1.0010,  ..., 1.0010, 1.0010, 1.0010],
        [1.0010, 1.0010, 1.0010,  ..., 1.0010, 1.0010, 1.0010],
        [1.0010, 1.0010, 1.0010,  ..., 1.0010, 1.0010, 1.0010]])


In [42]:
from models.model import make_model
from experiments.configs import get_config
config = get_config(16)
model, energy_fn = make_model(
    -torch.pi, torch.pi, **config.model['kwargs'])


c:\Users\继续革命\AppData\Local\Programs\Python\Python38\lib\site-packages\nflows\utils\torchutils.py:73: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\BatchLinearAlgebra.cpp:2497.)
  q, _ = torch.qr(x)


TypeError: partial is not a Module subclass

In [ ]:
import torch
from torch.nn import (Linear, Module)
class Tmp(Module):
    